# Analiza zboiru danych związanych z katastrofą Titanica
### Na podsawie poniższej analizy chciałbym zobaczyć i możliwie przewidzieć określić wpływ czynników związanych z przeżywalonością pasażerów statku

## 1. Określenie Problemu
### Zrozumienie wpływu czynników decydujących o przezprzywalności pasażerów podczas katastrofy Titanica.
* Czy istniała zależność czynników demofgraficznych takich jak wiek, płeć i statuś społeczny.
* Czy lokalizacja na statku na przykłąd kabina i pokład mogły mieć wpływ  